In [1]:
import datasetdatabase as dsdb
import pandas as pd
import quilt

In [2]:
mngr = dsdb.ConnectionManager(dsdb.LOCAL, user="jacksonb")
mngr

LOCAL:
	driver: sqlite
	link: /active/examples/local_database/local.db

In [3]:
local = mngr.connect(dsdb.LOCAL)
local

------------------------------- DATASET DATABASE -------------------------------
--------------------------------------------------------------------------------
User:
	rows: 0
	example: None
--------------------------------------------------------------------------------
Iota:
	rows: 0
	example: None
--------------------------------------------------------------------------------
SourceType:
	rows: 2
	example: {'SourceTypeId': 1, 'Name': 'FileSource', 'Description': 'Id attached should be read using FMS get.', 'Created': '2018-07-10 00:52:15.866332'}
--------------------------------------------------------------------------------
Source:
	rows: 0
	example: None
--------------------------------------------------------------------------------
FileSource:
	rows: 0
	example: None
--------------------------------------------------------------------------------
Dataset:
	rows: 0
	example: None
--------------------------------------------------------------------------------
IotaDatasetJuncti

In [4]:
quilt.ls()

/home/jovyan/.local/share/QuiltCli/quilt_packages


In [5]:
test = pd.DataFrame([{"hello": 1, "world": 2}, {"hello": 3, "world": 4}])
test

,hello,world
0,1,2
1,3,4


In [11]:
local.upload_dataset(test)

{'DatasetId': 1,
 'Name': 'fms_ae47a461a4ab791aab0fa25745ea846b',
 'Description': None,
 'SourceId': 1,
 'Created': '2018-07-10 00:52:16.569712'}

In [12]:
local

------------------------------- DATASET DATABASE -------------------------------
--------------------------------------------------------------------------------
User:
	rows: 1
	example: {'UserId': 1, 'Name': 'jacksonb', 'Description': None, 'Created': '2018-07-10 00:52:16.457849'}
--------------------------------------------------------------------------------
Iota:
	rows: 4
	example: {'IotaId': 1, 'GroupId': 0, 'Key': 'hello', 'Value': '1', 'ValueType': "<class 'numpy.int64'>", 'Created': '2018-07-10 00:52:16.505399'}
--------------------------------------------------------------------------------
SourceType:
	rows: 2
	example: {'SourceTypeId': 1, 'Name': 'FileSource', 'Description': 'Id attached should be read using FMS get.', 'Created': '2018-07-10 00:52:15.866332'}
--------------------------------------------------------------------------------
Source:
	rows: 1
	example: {'SourceId': 1, 'SourceTypeId': 1, 'Name': 'fms_ae47a461a4ab791aab0fa25745ea846b', 'Created': '2018-07-10 00:52

In [13]:
quilt.ls()

/home/jovyan/.local/share/QuiltCli/quilt_packages
dsdb_storage/fms_ae47a461a4ab791aab0fa25745ea846b latest               5a7e03c6dc595b962ab822ef86cfc88b94ac79593e41abadb4e8960ab70b2ad3
